In [ ]:
include("../src/project.jl")

In [ ]:
using Pkg

In [ ]:
import Pkg
Pkg.add("LanguageServer")

In [ ]:
Pkg.add("StatsBase")

In [ ]:
### definition of PosMatrix type

# struct PosMatrixCompressed{T, V, W}
#     name::String
#     modifications::Dict{String, Int}
#     nummods::Int
#     modfields::Int
#     numreads::Int
#     readfields::Int
#     chromindex::Dict{String, UnitRange{Int}}
#     chromrindex::Dict{Int, String}
#     readindex::V
#     readivs::IntervalCollection{W}
#     dataencoding::DataType
#     PM::T
# end

In [ ]:
PMS = load_smf_data("/Users/alexandrehefrenh2/University of Exeter/Gene Regulatory Defects in Disease - Documents/data/ont/posmatrix/")

In [ ]:
typeof(PMS[1])

In [ ]:
prod(size(PMS[1].readindex))*sizeof(Int64)/1024/1024 # prod is the product of all elements in the array 
# the output is the size of the readindex in MB

In [ ]:
prod(size(PMS[1].PM))*sizeof(UInt8)/1024/1024

In [ ]:
size(PMS[1].readindex)

In [ ]:
PMS[1].name

In [ ]:
#  how access first 10 reads of PMS[1].PM matrix
PMS[1].PM[1:10, :]

In [ ]:
typeof(PMS[1].readindex) # is a 2d array

In [ ]:
PMS[1].readindex .+ 1

In [ ]:
PMS[1].readindex[:, 1:10]

In [ ]:
PMS[1].readindex[:, 1:10]'
# transpose needed because data is stored in column order in PosMatrix

In [ ]:
#  PMS[1]: file PosMatrixCompressed
# PMS[1].readindex: matrix of position readings 
# PMS[1].PM: matrix of genomic psoitions and methylation probabilities

In [ ]:
readdf = DataFrame(PMS[1].readindex[:, 1:10]', [:chromstrand, :genomestart, :genomestop, :mod_6mA_start, :mod_6mA_stop, :mod_5mC_start, :mod_5mC_stop])
readdf.chrom = getindex.(Ref(PMS[1].chromrindex), last.(GenomeFragments.get_strand_chrom_enc.(readdf.chromstrand)))
readdf.strand = first.(GenomeFragments.get_strand_chrom_enc.(readdf.chromstrand))
readdf # DataFrame with first 10 reads and 9 columns

In [ ]:
PMS[1].modifications

In [ ]:
PMS[1].modifications["6mA"]

In [ ]:
PMS[1].readindex
# PM.readindex[rind[1], readindex]:PM.readindex[rind[2], readindex]

In [ ]:
PMS[1].readindex[:, 1] 

# note first 6mA modification occurs at 1 and goes to 123
#  from 124 to 1064 are 5mA modifications

In [ ]:
println(PMS[1].readindex[4, 1] )
println(PMS[1].readindex[5, 1] )

In [ ]:
# Assuming rind and readindex are defined and accessible
start_index = PMS[1].readindex[4, 1]
end_index = PMS[1].readindex[5, 1]

# Print the indices being accessed
println("Accessing indices from: ", start_index, " to ", end_index)

In [ ]:
PMS[1].readindex[:, 2]

# second read has 6mA modification from 1065 to 1464
# 5mA modification from 1465 to 2656

In [ ]:
PMS[1].readindex[:, 3]
# 3rd read has 6mA modification from 2654 to 3052
# 5mA modification from 3053 to 4430

In [ ]:
function get_mod_data(readindex, modification, PM) #PM is a PosMatrixCompressed object, ex: PMS[1]
    mi = PM.modifications[modification]
    off = (mi - 1)*2
    rind = off .+ 3 .+ (1:2)
    pind = PM.readindex[rind[1], readindex]:PM.readindex[rind[2], readindex]
    d = decompress(Int32, PM.PM[pind])
    reshape(d, 2, div(length(d), 2))
end

In [ ]:
# Adjusted function
function get_mod_data(read_value, modification, PM) #PM is a PosMatrixCompressed object, ex: PMS[1]
    mi = PM.modifications[modification] # Get the modification index
    off = (mi - 1)*2  #multiplying by 2: `off` points to the correct pair of columns for the specified modification.
    rind = off .+ 3 .+ (1:2) # Calculate row indices for start and stop positions. (1:2) because two values are needed
                             #   +3 to skip the first three columns: chromstrand, genomestart, genomestop
    pind = PM.readindex[rind[1], read_value]:PM.readindex[rind[2], read_value] #start/stop positions of a modification on a specific read
    d = decompress(Int32, PM.PM[pind])
    reshape(d, 2, div(length(d), 2))
end

# Adding (1:2): This creates a pair of consecutive indices. 
# The result of off + 3 gives a base index, and adding (1:2) (which is [1, 2]) provides 
# the two indices that are used to index into the readindex array.

In [ ]:
# Modified dataframe to include all columns from PMS[1].readindex
read_df = DataFrame(PMS[1].readindex', [:chromstrand, :genomestart, :genomestop, :mod_6mA_start, :mod_6mA_stop, :mod_5mC_start, :mod_5mC_stop])

# Assuming the rest of the code remains the same for adding 'chrom' and 'strand' columns
read_df.chrom = getindex.(Ref(PMS[1].chromrindex), last.(GenomeFragments.get_strand_chrom_enc.(read_df.chromstrand)))
read_df.strand = first.(GenomeFragments.get_strand_chrom_enc.(read_df.chromstrand))

# Display the DataFrame
read_df # DataFrame with all reads and their respective columns

# Accessing mod_6mA_start and mod_6mA_stop from readdf for the first row as an example
mod_6mA_start_value = read_df.mod_6mA_start[1]
mod_6mA_stop_value = read_df.mod_6mA_stop[1]

# Use these variables as needed
println("mod_6mA_start for the first read is: ", mod_6mA_start_value)
println("mod_6mA_stop for the first read is: ", mod_6mA_stop_value)

In [ ]:
# head of the read_df
read_df[1:22, :]

In [ ]:
modification_name = "6mA"
modification_start_col = "mod_$(modification_name)_start"
read_df[1:5, Symbol(modification_start_col)]

# equivalent in python is read_df.loc[1, f"mod_{modification_name}_start"]

In [ ]:
# To work with dataframe 

function get_mod_datadf(read_index, modification_name, file)
    # Dynamically construct column names for start and stop positions
    modification_start_col = "mod_" * modification_name * "_start"
    modification_stop_col = "mod_" * modification_name * "_stop"

    # Extract start and stop positions from the DataFrame for the specified read
    start_pos = read_df[read_index, Symbol(modification_start_col)]
    stop_pos = read_df[read_index, Symbol(modification_stop_col)]

    # Decompress data between start and stop positions
    d = decompress(Int32, file.PM[start_pos:stop_pos])

    # Reshape the decompressed data into a 2-array format
    reshaped_data = reshape(d, 2, div(length(d), 2))

    

    return reshape(d, 2, div(length(d), 2)) 
end


# Example 

# print each row of the data on a separate line
function print_mod_data(read_index, modification_name, PM)
    mod_data = get_mod_datadf(read_index, modification_name, PM)
    for row in eachrow(mod_data)
        println(row)
    end
end

# To get and print the 6mA modification data for the first read
print_mod_data(1, "6mA", PMS[1])
# To get and print the 6mA modification data for the second read
print_mod_data(2, "6mA", PMS[1])
# To get and print the 5mC modification data for the first read
print_mod_data(1, "5mC", PMS[1])



get_mod_datadf(5, "6mA", PMS[1])

In [ ]:
# Convert the reshaped data into a DataFrame
firstreading_df = DataFrame((get_mod_datadf(1, "6mA", PMS[1]))', [:Position, :Modification_Probability])


# Get the first 5 rows of the DataFrame
head_df = first(firstreading_df, 5)

# Get the last 5 rows of the DataFrame
# tail_df = last(reading_df, 5)

In [ ]:
# # All reads Dataframe
# using DataFrames

# # Initialize an array to store the data for all readings
# all_readings_data = []

# # Total number of readings
# N = size(PMS[1].readindex, 2)

# for i in 1:N
#     # Retrieve data for the current reading
#     current_data = get_mod_datadf(i, "6mA", PMS[1])
    
#     # Convert the data into a DataFrame format (or keep as an array of tuples)
#     current_df_data = [(pos, prob) for (pos, prob) in zip(current_data[1, :], current_data[2, :])]
    
#     # Append the current data to the all_readings_data array
#     append!(all_readings_data, current_df_data)
# end

# # Convert the collected data into a DataFrame in one go
# all_readings_df = DataFrame(all_readings_data, [:Position, :Modification_Probability])

# all_readings_df
# # Now, all_readings_df contains data from all readings

In [ ]:
# # Plot the histogram for the Modification_Probability column
# histogram(all_readings_df.Modification_Probability, bins=257, xlabel="Modification Probability", ylabel="Frequency", title="Histogram of Modification Probabilities")

In [ ]:
## All reads Dataframe

# # Pre-allocate an array 
# all_readings = Vector{Vector{Tuple{Int32, Int32}}}(undef, N)

# # Total number of readings
# N = size(PMS[1].readindex, 2)

# for i in 1:N
#     current_data = get_mod_datadf(i, "6mA", PMS[1])
#     current_df_data = [(pos, prob) for (pos, prob) in zip(current_data[1, :], current_data[2, :])]
#     all_readings[i] = current_df_data
# end


# # Flatten the vector of vectors of tuples into a single vector of tuples
# flattened_readings = reduce(vcat, all_readings)

# # Convert the flattened vector of tuples into a DataFrame
# all_readings_df2 = DataFrame(flattened_readings, [:Position, :Modification_Probability])

# all_readings_df2

In [ ]:
function get_mod_datadf2(read_index, start_col_name, df, file)
    # Find the index of the start column
    # if start_col_name ∉ names(df)
    #     error("Column name '$start_col_name' not found in DataFrame.")
    # end
    start_col_index = findfirst(isequal(start_col_name), names(df)) # findfirst return the index of the first element satisfying condition


    # Assume the stop column is the next column in the DataFrame
    stop_col_index = start_col_index + 1

    # Retrieve the actual column names from their indices
    start_pos_col = names(df)[start_col_index]
    stop_pos_col = names(df)[stop_col_index]

    # Extract start and stop positions from the DataFrame for the specified read
    start_pos = df[read_index, start_pos_col]
    stop_pos = df[read_index, stop_pos_col]

    # Decompress data between start and stop positions
    d = decompress(Int32, file.PM[start_pos:stop_pos])

    # Reshape the decompressed data into a 2-array format
    return reshape(d, 2, div(length(d), 2))
end

# Assuming print_mod_data is defined correctly elsewhere and PMS[1] is the correct DataFrame
get_mod_datadf2(1, "mod_6mA_start", read_df, PMS[1])

In [ ]:
get_mod_datadf(1, "6mA", PMS[1])

In [ ]:
data_6mA = get_mod_data(1, "6mA", PMS[1])

In [ ]:
data_6mA = get_mod_data(2, "6mA", PMS[1])

In [ ]:
data_5mC = get_mod_data(1, "5mC", PMS[1])

In [ ]:
# Scatter plot of the 6mA and 5mC modification data
plot()
scatter!(data_6mA[1, :], data_6mA[2, :]./256, lab="6mA")
scatter!(data_5mC[1, :], data_5mC[2, :]./256, lab="5mC")


In [ ]:
## first row is genomic position
## second row is the probability that the sequencer assigned to base being modified

data_6ma = reshape(decompress(Int32, PMS[1].PM[1:123]), 2, 30)

In [ ]:
# values become probability after divinding by 256
data_6ma[2, 1] / 256

In [ ]:
bar(data_6ma[1, :], data_6ma[2, :]./256)

### Histogram 

To calculate a histogram of the modification probabilities:

1. Iterate over each read in the PM object
2. For each read, use the get_mod_datadf function to get the modification data on the second row
3. Extract the second row of the matrix returned by get_mod_datadf, which contains the modification probabilities

5. Update the histogram count
4. Normalise histogram by dividing  each bin's count by the total count of all bins

Note: choose "raw count" or `realative frequency` (normalised)


Recall: Direct binning Vs normalised binning  

Direct Binning: If we have integer values strictly between 0 and 256, handling edge cases is generally not needed unless there's a possibility of having exactly 256 due to some data processing.  

Normalized Binning: When normalizing values first, we have to handle edge cases correctly to ensure all values are binned properly.

In [ ]:

function calculate_modification_histogram(file, modification)
    # Initialize a histogram with 257 bins (0 to 256 inclusive)
    histogram = zeros(Int, 257)
    
    # Total number of reads is the number of columns in file.readindex
    total_reads = size(file.readindex, 2)
    
    for read_index = 1:total_reads
        # Get modification data for the current read
        mod_data = get_mod_datadf(read_index, modification, file)
        
        # Extract the second row (modification probabilities)
        mod_probs = mod_data[2, :] 
        
        # # Update the histogram
        # for prob in mod_probs  #prob are integers 
        #     histogram[prob + 1] += 1  # prob +1 because Julia arrays are 1-indexed
        # end

         # update the histogram using countmap to avoid looping over all values
         counts = countmap(mod_probs) # countmap: returns a dictionary with the counts of each unique value
         for (prob, count) in counts
             histogram[prob + 1] += count   #prob are integers starting from 0
         end
    end
     
    # Normalise the histogram
    total_counts = sum(histogram)
    normalized_histogram = histogram / total_counts
    
    return histogram , normalized_histogram
end

hist, norm_hist = calculate_modification_histogram(PMS[1], "6mA")
println("Histogram: ", hist)
println("Normalized histogram: ", norm_hist)

In [ ]:
# If using a different number of bins, such as 30 instead of 257, 
#  we need to scale the original values (0 to 256) into the specified number of bins

function calculate_modification_histogram2(file, modification)
    # Initialize a histogram with a different number of bins from the avaialble 257 values
    # histogram = zeros(Int, 30) # 30 bins
    histogram = fill(0,30)
    
    # Total number of reads is the number of columns in PM.readindex
    total_reads = size(file.readindex, 2)
    
    for read_index = 1:total_reads
        # Get modification data for the current read
        mod_data = get_mod_data(read_index, modification, file)
        
        # Extract the second row (modification probabilities) and normalize
        mod_probs = mod_data[2, :] ./ 256 # dividing by 256, resulting in values between 0 and 1.
        
        # Update the histogram
        for prob in mod_probs  # prob ranges from 0 to 1  (not integer values)
            # Scale to 0-29 range and round down for bin index
            bin_index = if prob == 1.0
                30  # If prob is exactly 1, place it in the last bin
            else
                Int(floor(prob * 30)) + 1  # +1 for 1-indexing
            end
            
            histogram[bin_index] += 1
        end
    end
    
    histogram
    # normalise the histogram
    
    total_counts = sum(histogram)
    histogram_normalized = histogram / total_counts

    return histogram, histogram_normalized
end

raw_counts, normalised_counts =  calculate_modification_histogram2(PMS[1], "6mA")
println("raw Counts: ", raw_counts)
println("Normalised counts: ", normalised_counts)

For pre-binned data, where we already have the counts for each bin (like normalised_counts), and especially if these counts are normalized, we would need the bin edges to accurately plot the histogram, as the histogram() function won't inherently know how the data was binned.

If we want to plot pre-binned data with Plots.jl, we should use a different approach. Since we have normalized counts, we need to plot them against the bin centers or edges directly using a *bar plot* or a *step plot *


In [ ]:
using Plots

hist, norm_hist = calculate_modification_histogram(PMS[1], "6mA")
# Bin centres from the edges of the bins
bin_edges = range(0, stop=1, length=258)  # 256 bins, thus 258 edges
bin_centers = (bin_edges[1:end-1] .+ bin_edges[2:end]) ./ 2

# Now plot using bar, since data is already binned and normalised
bar(bin_centers, norm_hist, xlabel="Modification Probability", ylabel="Frequency", title="Normalised Modification Probabilities", legend=false, color=:blue)

In [ ]:
# Plot the raw counts histogram for 6mA modification
plot(1:257, hist, label="6mA", xlabel="Bin", ylabel="Frequency", title="Raw Counts")

In [ ]:
# ALTERNATIVELY

# Function to create a histogram with a specified number of bins
"""Create a histogram with a specified number of bins for a given modification type."""
function create_histogram(modification, file, desired_bins)
    # Initialize the histogram with the desired number of bins
    histogram = zeros(Int, desired_bins)

    # Iterate over all reads
    for read_index in 1:size(file.readindex, 2)
        # Get modification data for the current read
        mod_data = get_mod_datadf(read_index, modification, file)

        # Extract the second row (modification probabilities)
        mod_probs = mod_data[2, :]  # Assume mod_probs are integers

        # Rescale probabilities to fit into the desired number of bins
        for prob in mod_probs
            bin_index = floor(Int, (prob / 256) * (desired_bins - 1)) + 1
            histogram[bin_index] += 1
        end
    end

    # Normalize the histogram
    total_counts = sum(histogram)
    normalized_histogram = histogram / total_counts

    return normalized_histogram
end

# Example
hist6ma = create_histogram( "6mA", PMS[1], 257)

desired_bins = 257
# Plot the histogram using the computed data
bin_edges = range(0.0, stop=1.0, length=desired_bins + 1)
bin_centers = [(bin_edges[i] + bin_edges[i+1]) / 2 for i in 1:length(bin_edges)-1]

plot(bin_centers, hist6ma, seriestype=:bar, xlabel="Modification Probability", ylabel="Frequency", title="Modification Probability Histogram", label="6mA", color=:blue)

Using `fit()` function to fit a distribution to the data

The `fit` method from the `StatsBase` package does not normalise the histogram counts by default. It calculates the counts of elements falling into each bin based on the provided data (all_mod_probs) and the bin edges (bin_edges). The result is the raw counts of elements in each bin, thesea re stored into histogram.weights. For normalised histogram, a PDF, the total area (under the histogram) sums up to one

In [ ]:
# Histogram for PDF of "modification probabilities" 
using StatsBase

# Dynamic binning
function calculate_modification_fithistogram(file, modification)
    # Initialize an empty array to collect all modification probabilities
    all_mod_probs = Float64[]

    # Total number of reads is the number of columns in PM.readindex
    total_reads = size(file.readindex, 2)

    for read_index = 1:total_reads
        # Get modification data for the current read
        mod_data = get_mod_datadf(read_index, modification, file)

        # Extract the second row (modification probabilities) and append to all_mod_probs
        append!(all_mod_probs, mod_data[2, :])
    end

    # Determine the range for the histogram bins
    min_prob = minimum(all_mod_probs)
    max_prob = maximum(all_mod_probs)
    bin_edges = range(min_prob, max_prob, length=258) # 257 bins means 258 edges
    bin_centers = (bin_edges[1:end-1] .+ bin_edges[2:end]) ./ 2  # bin centers


    # Create a histogram with 10 bins (dynamic binning)
    histogram = fit(Histogram, all_mod_probs, bin_edges, closed=:right) # closed=:right means right-closed intervals

    # Return the bin counts from the histogram
    histogram.weights 


    # Assuming histogram is the result from the fit method
    total_counts = sum(histogram.weights)
    # Calculate a single individual bin width
    bin_width = bin_edges[2] - bin_edges[1] # Assuming equal-width bins
    # bin_widths = diff(bin_edges) # For variable-width bins
    normalized_counts = histogram.weights / (total_counts * bin_width)
    
    # assert if the area adds to 1.0
    @assert sum(normalized_counts .*bin_width) ≈ 1.0 # Check if the sum of normalized counts is 1.0
    # println("Sum of normalized counts: ", sum(normalized_counts))
    # println("bin width: ", bin_width)
    # println("value", sum(normalized_counts .* bin_width))

    return histogram.weights, normalized_counts, bin_edges, bin_centers
end

raw_counts, normalized_counts, bin_edges, bin_centers = calculate_modification_fithistogram(PMS[1], "6mA")

println("Raw counts: ", raw_counts)
println("Normalized counts: ", normalized_counts)


In [ ]:
# Plotting the histogram
plot(1:257, normalized_counts, xlabel="Bin Index", ylabel="density", lab="6mA", color=:blue)

In [ ]:
normalized_counts[1:3]

In [ ]:
# Calculate the cumulative distribution 
cumulative_distribution = cumsum(normalized_counts)

# Find the bin where the cumulative distribution first exceeds 50%
bin_index = findfirst(x -> x > 0.5, cumulative_distribution)

println("Bin index for 50% CDF = " ,bin_index, " with value ", cumulative_distribution[bin_index])

println(round( cumulative_distribution[bin_index]*100, digits=2) , "% of the data is below ", (bin_index)/256 , " probability of modification" )
# third bin (index=3) has  upper bound of 3/256 = 0.01171875 or 1.17% probability of modification

println("Methylation probability at this index is ", normalized_counts[bin_index]) #calculate the value of the bin at the index 

In [ ]:
# Plotting the histogram
bar(bin_centers, normalized_counts, xlabel="Bin Index", ylabel="Density", lab="6mA", color=:blue)

In [ ]:
# calculate the half quantile of the modification probabilities

half_quantile = quantile(normalized_counts, 0.5)
# 50% of the modification probabilities are below this value


with Plots:

In [ ]:
# using Plots  

# 257 bins for 256 modification probabilities
function plot_modification_histogram(file, modification)
    # Initialize an empty array to collect all modification probabilities
    all_mod_probs = Float64[]

    # Total number of reads is the number of columns in PM.readindex
    total_reads = size(file.readindex, 2)

    for read_index = 1:total_reads
        # Get modification data for the current read
        mod_data = get_mod_datadf(read_index, modification, file)

        # Extract the second row (modification probabilities)
        mod_probs = mod_data[2, :]

        # Append the probabilities to the all_mod_probs array
        append!(all_mod_probs, mod_probs)
    end

    # Plot the histogram of all modification probabilities
    histogram(all_mod_probs, bins=257, xlabel="Modification Probability", ylabel="Density", 
    title="Histogram", label="6mA" , normalize=true)
    # normalize=true normalizes to form a PDF 
end

In [ ]:
plot_modification_histogram(PMS[1], "6mA")

In [ ]:
plot_modification_histogram(PMS[1], "5mC")

In the original version, we are dealing with modification probabilities that are integers in the range [0, 256]. We can directly use these integer values as indices to update the histogram. The countmap() function is used to count the frequency of each unique value in mod_probs, and then we update the histogram accordingly.

However, when we calculate the mean methylation, the resulting values are not integers. They are floats between 0 and 1 (after dividing by 256). We cannot use these floating-point numbers directly as indices to update the histogram.

In [ ]:
mod_data1 = get_mod_datadf(3, "6mA", PMS[1])

In [ ]:
size(read_df ,1 )  == size(PMS[1].readindex,2)

In [ ]:
data = get_mod_datadf(22, "6mA", PMS[1])

if !isempty(data)  # Correct way to check if data is not empty
    data_mean = mean(data)
    println("Mean of the data is: ", data_mean)
else
    data_mean = mean(data)
    println("Data is empty.")
    println("Mean of the data is: ", data_mean)
end

In [ ]:
# Assuming mod_data[2, :] is the target slice
mod_data_22= get_mod_datadf(22, "6mA", PMS[1])[2, :]

# Check if any element in the slice is NaN
contains_nan = any(isnan.(mod_data_22))

println("Contains NaN: ", contains_nan)
println("mean of the reading is ", mean(mod_data_22))


In [ ]:
get_mod_datadf(180, "5mC", PMS[1])[2, :]

In [ ]:
using StatsBase

function calculate_mean_methylation_histogram(file, modification)
    # Initialize an empty array to collect mean methylation values
    # mean_methylations = Float64[]
    mean_methylations = zeros(Float64, size(PMS[1].readindex,2) ) # Allocate a vector of size number of reads

    # Total number of reads is the number of columns in PM.readindex
    total_reads = size(file.readindex, 2)

    for read_index = 1:total_reads
        # Get modification data for the current read
        mod_data = get_mod_datadf(read_index, modification, file)

        # Calculate mean methylation (average of modification probabilities)
        mean_methylation = mean( mod_data[2, :] ./256 ) 
        # push!(mean_methylations, mean_methylation)
        mean_methylations[read_index] = mean_methylation # ttore the mean
    end

    # Determine the range for the histogram bins
    bin_edges = range(0.0, stop=1.0, length=257) # 256 bins for [0, 1]

    # Create a histogram
    hist = fit(Histogram, mean_methylations, bin_edges, closed=:left)
   

    return hist
end


In [ ]:
Pkg.add("OnlineStats")

In [ ]:
# For comparison with the previous implementation
using OnlineStats

function calculate_mean_methylation_histogram2(file, modification)
    # Initialize an empty array to collect mean methylation values
    mean_methylations = zeros(Float64, size(file.readindex, 2)) # Allocate a vector of size number of reads

    # Total number of reads is the number of columns in file.readindex
    total_reads = size(file.readindex, 2)

    for read_index = 1:total_reads
        # Get modification data for the current read
        mod_data = get_mod_datadf(read_index, modification, file)

        # Calculate mean methylation (average of modification probabilities)
        mean_methylation = mean(mod_data[2, :] ./ 256)
        mean_methylations[read_index] = mean_methylation # Store the mean
    end

    # Create a histogram using OnlineStats
    # specify the range (0.0 to 1.0) and the number of bins (256)
    histogram = Hist(0.0: 1/256: 1.0) # Hist object from OnlineStats

    # Fit the histogram with the mean_methylations data
    fit!(histogram, mean_methylations) #fit!  from OnlineStats
    # After calculating mean_methylations
    println("Min mean methylation: ", minimum(mean_methylations))
    println("Max mean methylation: ", maximum(mean_methylations))

# Continue to create and fit the histogram

    return histogram
end


hist_data6ma = calculate_mean_methylation_histogram2(PMS[1], "6mA")
# plot the histogram
# histogram(hist_data6ma,  xlabel="Mean Methylation", ylabel="Frequency", title="Histogram of Mean Methylation")
plot(hist_data6ma,  label = "6mA", color = :blue) 


In [ ]:
# Find empty data for a modification 6ma or 5mC -

extracted_data = []
empty_data_indices = []  # List to keep track of indices with empty data

# Assuming PMS[1] is defined elsewhere in your code
# Loop through a range of j values. Adjust the range as necessary.
for j in 1:size(PMS[1].readindex, 2)
    try
        # Attempt to extract data for the current value of j
        data = get_mod_datadf(j, "6mA", PMS[1])[2,:]
        
        # Check if the data is an empty matrix
        if size(data, 1) == 0 || size(data, 2) == 0  # "OR" 
            println("Found empty data at j = $j")
            push!(empty_data_indices, j)  # Add the index to the list
            continue  # Skip to the next iteration
        end
        
        # If data extraction is successful and not empty, append the data to the extracted_data list
        push!(extracted_data, data)
    catch e
        # If an error occurs, print the error and continue to the next iteration
        println("Stopped at j = $j due to error: $e")
        continue
    end
end


# # If there are any indices with empty data, print them
# if !isempty(empty_data_indices)
#     println("Indices with empty data: ", join(empty_data_indices, ", "))
# else
#     println("Data extraction completed without encountering missing or empty values.")
# end

In [ ]:
# For read_index = 155
println(get_mod_datadf(155, "6mA", PMS[1])[2,:])
println(get_mod_datadf(155, "5mC", PMS[1])[2,:])  

In [ ]:
# index 22 - another example of empty data for 6mA
get_mod_datadf(22, "6mA", PMS[1])

In [ ]:
get_mod_datadf(1, "6mA", PMS[1])
get_mod_datadf(1, "5mC", PMS[1])

In [ ]:
# Mean methylation histogram for 6mA and 5mC modifications

hist_data6ma = calculate_mean_methylation_histogram(PMS[1], "6mA")
hist_data5mc = calculate_mean_methylation_histogram(PMS[1], "5mC")

In [ ]:
# typeof(hist_data6ma.edges[1])

In [ ]:
bin_counts = hist_data6ma.weights #
bin_edges = hist_data6ma.edges[1] # edges of the bins for 1st dimension (one-dimensional histogram)
bin_centers = (bin_edges[1:end-1] .+ bin_edges[2:end]) ./ 2

# Calculate the width assuming equal-width bins
bar_width = bin_edges[2] - bin_edges[1]


bar(0:256, bin_counts, width=bar_width, xlabel="Bins", ylabel="Frequency", title="Mean Methylation", label="6mA", color=:blue)
# bar(bin_centers, bin_counts, width=(bin_edges[2]-bin_edges[1]), xlabel="Modification Probability", ylabel="Density", title="Modification Probability Histogram", label="6mA")


In [ ]:
bin_counts = hist_data5mc.weights #
bin_edges = hist_data5mc.edges[1] # edges of the bins for 1st dimension (one-dimensional histogram)
bin_centers = (bin_edges[1:end-1] .+ bin_edges[2:end]) ./ 2

# Calculate the width assuming equal-width bins
bar_width = bin_edges[2] - bin_edges[1]


# bar(0:256, bin_counts, width=bar_width, xlabel="Bin", ylabel="Density", title="Modification Probability Histogram", label="6mA", color=:blue)
bar(bin_centers, bin_counts, width=bar_width, xlabel="Bins", ylabel="Density", title="Mean Methylation", label="5mC", color=:blue)


### Distribution of distance between methylated bases



In [ ]:
# Function to calculate the distribution of the distance between bases methylated with probability > t



function calc_dist_methylated_bases(read_df, modification_name, file, t)
    dists = []
    for read_index in 1:size(read_df, 1)
        mod_data = get_mod_datadf(read_index, modification_name, file)
        methylated_positions = mod_data[1, mod_data[2, :] .> t]
        push!(dists, diff(methylated_positions))
    end
    return dists
end

### Load peakmotif file 

In [ ]:
peakmotif = load_tf_data("/Users/alexandrehefrenh2//University of Exeter/Gene Regulatory Defects in Disease - Documents/data/islets/peakmotif/")

In [ ]:
peakmotif["CTCF"]

In [ ]:
dw = 1000
xp = -dw:dw
cls = @with peakmotif["CTCF"] (:chrom, [div(s + e, 2) .+ (-dw:dw) for (s, e) in zip(:motifstart, :motifstop)], :strand)
ctcf_ivs = IntervalCollection(Interval.(cls[1], cls[2], first.(cls[3]), 1:length(cls[1])));

In [ ]:
ctcf_ivs

In [ ]:
mean_6ma = modmeta(ctcf_ivs, PMS[1], "6mA");
mean_5mC = modmeta(ctcf_ivs, PMS[1], "5mC");

In [ ]:
zscore(x)  = (x .- mean(x))./std(x)

In [ ]:
plot()
plot!(xp, mean_6ma[1], lab="6mA")
plot!(xp, mean_5mC[1], lab="5mC")

In [ ]:
plot()
plot!(xp, zscore(mean_6ma[1]), lab="6mA")
plot!(xp, zscore(mean_5mC[1]), lab="5mC")

In [ ]:
qc_6mA = SMFTools.modmetaqc(ctcf_ivs, PMS[1], "6mA")
qc_5mC = SMFTools.modmetaqc(ctcf_ivs, PMS[1], "5mC")

In [ ]:
heatmap(-dw:dw, range(0, 1, length=256), qc_6mA[1]', clims=(0, 500), ylabel="Basecaller methylation probability", xlabel="CTCF", title="Frequency of modified base calls")

In [ ]:
heatmap(-dw:dw, range(0, 1, length=256), qc_5mC[1]', clims=(0, 100), ylabel="Basecaller methylation probability", xlabel="CTCF")

In [ ]:
mods_6mA = modheat(ctcf_ivs, PMS[1], "6mA")

## Third part

Export data 

In [ ]:
Pkg.add("HDF5")

In [ ]:
using HDF5, DataFrames

# Extract the data as arrays (assuming your DataFrame has columns Position and Modification_Probability)
positions = firstreading_df.Position
probabilities = firstreading_df.Modification_Probability

# Open an HDF5 file for writing
h5open("data.h5", "w") do file
    # Create datasets for positions and probabilities
    write(file, "positions", positions)
    write(file, "probabilities", probabilities)
end